<a href="https://colab.research.google.com/github/RQLuo/MixTeX/blob/main/MixTex_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Import and Inference

In [1]:
from transformers import AutoTokenizer, VisionEncoderDecoderModel, AutoImageProcessor
from PIL import Image
import requests

feature_extractor = AutoImageProcessor.from_pretrained("MixTex/ZhEn-Latex-OCR")
tokenizer = AutoTokenizer.from_pretrained("MixTex/ZhEn-Latex-OCR", max_len=296)
model = VisionEncoderDecoderModel.from_pretrained("MixTex/ZhEn-Latex-OCR")
#imgen = Image.open(requests.get('https://cdn-uploads.huggingface.co/production/uploads/62dbaade36292040577d2d4f/eOAym7FZDsjic_8ptsC-H.png', stream=True).raw)
#print(tokenizer.decode(model.generate(feature_extractor(imgen, return_tensors="pt").pixel_values)[0]).replace('\\[','\\begin{align*}').replace('\\]','\\end{align*}'))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Dataset import and model training

In [ ]:
!pip install datasets accelerate

In [2]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset
from PIL import Image
import torch

dataframe = load_dataset("MixTex/Pseudo-Latex-ZhEn-1")

In [3]:
from torch.utils.data import Dataset
class MixTexDataset(Dataset):
    def __init__(self, dataframe, tokenizer, feature_extractor, max_length=256):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.dataframe['train'])

    def __getitem__(self, idx):
        image = self.dataframe['train'][idx]['image'].convert("RGB")
        target_text = self.dataframe['train'][idx]['text']
        pixel_values = self.feature_extractor(image, return_tensors="pt").pixel_values
        target = self.tokenizer(target_text, padding="max_length", max_length=self.max_length, truncation=True).input_ids
        labels = [label if label != self.tokenizer.pad_token_id else -100 for label in target]
        return {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
traindataset = MixTexDataset(dataframe, tokenizer, feature_extractor= feature_extractor)

In [4]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=12,
    predict_with_generate=True,
    logging_dir='./logs',
    learning_rate=5e-5,
    save_total_limit=1,
    logging_steps=100,
    save_steps=500,
    num_train_epochs=3,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=traindataset,
)

In [ ]:
trainer.train()

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Step,Training Loss
100,0.195300
